In [1]:
import pandas as pd
import glob
from sklearn.metrics import classification_report, precision_recall_fscore_support

from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F

In [2]:
%%time
metrics = {
    'task': [], 'task_lang': [], 'prompt_lang': [], 'prompt': [], 'model': [], 'acc': [],
    'macro_f1': [], 'macro_prec': [], 'macro_rec': [],
    'micro_f1': [], 'micro_prec': [], 'micro_rec': [],
    'weighted_f1': [], 'weighted_prec': [], 'weighted_rec': [],
}
for path in glob.glob('../outputs/*.csv'):
    if '_nusantara_text_' in path:
        task, meta = path.split('/')[-1].split('_nusantara_text_')
        try:
            prompt, model = meta[:-4].split('_')
        except:
            prompt, model, rehearse = meta[:-4].split('_')
            model = f'{model}-{rehearse}'
    elif  '_nusantara_pairs_' in path:
        task, meta = path.split('/')[-1].split('_nusantara_pairs_')
        try:
            prompt, model = meta[:-4].split('_')
        except:
            prompt, model, rehearse = meta[:-4].split('_')
            model = f'{model}-{rehearse}'
    elif 'nusa_kalimat_' in path:
        try:
            task, lang, prompt, model = path.split('/')[-1][:-4].split('nusa_kalimat_')[1].split('_')
        except:
            task, lang, prompt, model, rehearse = path.split('/')[-1][:-4].split('nusa_kalimat_')[1].split('_')
            model = f'{model}-{rehearse}'
        task = f'nusa_kalimat_{task}_{lang}'
    elif 'xnli' in path:
        try:
            task, task_lang, prompt, model = path.split('/')[-1][:-4].split('_')
        except:
            task, task_lang, prompt, model, rehearse = path.split('/')[-1][:-4].split('_')
            model = f'{model}-{rehearse}'
        task = f'{task}_{task_lang}'
    
    df = pd.read_csv(path)
    cls_report = classification_report(df['Gold'], df['Pred'], output_dict=True, zero_division=0)
    micro_f1, micro_prec, micro_rec, _ = precision_recall_fscore_support(df['Gold'], df['Pred'], average='micro')

    metrics['task'].append(task)
    metrics['task_lang'].append(task.split('_')[-1])
    metrics['prompt_lang'].append(prompt[:2])
    metrics['prompt'].append(prompt)
    metrics['model'].append(model)
    metrics['acc'].append(cls_report['accuracy'])
    metrics['macro_f1'].append(cls_report['macro avg']['f1-score'])
    metrics['macro_prec'].append(cls_report['macro avg']['precision'])
    metrics['macro_rec'].append(cls_report['macro avg']['recall'])
    metrics['micro_f1'].append(micro_f1)
    metrics['micro_prec'].append(micro_prec)
    metrics['micro_rec'].append(micro_rec)
    metrics['weighted_f1'].append(cls_report['weighted avg']['f1-score'])
    metrics['weighted_prec'].append(cls_report['weighted avg']['precision'])
    metrics['weighted_rec'].append(cls_report['weighted avg']['recall'])

CPU times: user 1min 36s, sys: 10.1 s, total: 1min 47s
Wall time: 2min 37s


In [3]:
mdf = pd.DataFrame(metrics)
mdf.groupby(['model','prompt_lang']).size()

model                     prompt_lang
bilingual-bloomz-560m     EN             165
                          ID             120
bloomz-1b1                EN             165
                          ID             120
bloomz-1b7                EN             165
                          ID             120
bloomz-3b                 EN             165
                          ID             120
bloomz-560m               EN             165
                          ID             120
checkpoint-19480          EN             114
                          ID             114
checkpoint-29220          EN             114
                          ID             114
checkpoint-38960          EN             114
                          ID             114
checkpoint-9740           EN             114
                          ID             114
monolingual-bloomz-560m   EN             165
                          ID             120
pair-bloomz-1b1           EN             120
                 

In [4]:
mdf.loc[
    (~mdf['task'].str.startswith('xnli')) & ~mdf['task'].isin(['su_emot', 'indonli', 'indolem_sentiment', 'smsa', 'emot', 'imdb_jv'])
,:].groupby(['model','prompt_lang']).size()

model                     prompt_lang
bilingual-bloomz-560m     EN             102
                          ID             102
bloomz-1b1                EN             102
                          ID             102
bloomz-1b7                EN             102
                          ID             102
bloomz-3b                 EN             102
                          ID             102
bloomz-560m               EN             102
                          ID             102
checkpoint-19480          EN             102
                          ID             102
checkpoint-29220          EN             102
                          ID             102
checkpoint-38960          EN             102
                          ID             102
checkpoint-9740           EN             102
                          ID             102
monolingual-bloomz-560m   EN             102
                          ID             102
pair-bloomz-1b1           EN             102
                 

In [5]:
mdf.loc[
    (~mdf['task'].str.startswith('xnli')) & ~mdf['task'].isin(['su_emot', 'indonli', 'indolem_sentiment', 'smsa', 'emot', 'imdb_jv'])
].to_csv('raw_result.csv', index=False)

In [6]:
mdf.loc[mdf['task'].str.startswith('nusa'),:].groupby(['prompt_lang', 'model'])[['acc','macro_f1','micro_f1','weighted_f1']].mean()

acc  macro_f1  micro_f1  \
prompt_lang model                                                    
EN          bilingual-bloomz-560m     0.412686  0.296968  0.412686   
            bloomz-1b1                0.437423  0.336300  0.437423   
            bloomz-1b7                0.479848  0.330984  0.479848   
            bloomz-3b                 0.494082  0.355584  0.494082   
            bloomz-560m               0.377802  0.281763  0.377802   
            checkpoint-19480          0.341641  0.226884  0.341641   
            checkpoint-29220          0.357450  0.256799  0.357450   
            checkpoint-38960          0.341225  0.238079  0.341225   
            checkpoint-9740           0.378706  0.267356  0.378706   
            monolingual-bloomz-560m   0.396675  0.286992  0.396675   
            pair-bloomz-1b1           0.428960  0.246121  0.428960   
            pair-bloomz-1b1-R-100     0.317824  0.237787  0.317824   
            pair-bloomz-1b1-R-1000    0.285673  0.205301  0.285673   
            pair-bloomz-1b1-R-10000   0.357011  0.211001  0.357011   
            pair-bloomz-560m          0.420639  0.271929  0.420639   
            pair-bloomz-560m-R-100    0.342018  0.236022  0.342018   
            pair-bloomz-560m-R-1000   0.400386  0.274475  0.400386   
            pair-bloomz-560m-R-10000  0.367112  0.274859  0.367112   
            random-bloomz-560m        0.391264  0.291142  0.391264   
            translation-bloomz-560m   0.311096  0.190644  0.311096   
ID          bilingual-bloomz-560m     0.319361  0.248532  0.319361   
            bloomz-1b1                0.449816  0.331199  0.449816   
            bloomz-1b7                0.428096  0.342054  0.428096   
            bloomz-3b                 0.500694  0.404762  0.500694   
            bloomz-560m               0.368551  0.309090  0.368551   
            checkpoint-19480          0.312491  0.208943  0.312491   
            checkpoint-29220          0.293319  0.204767  0.293319   
            checkpoint-38960          0.309924  0.244571  0.309924   
            checkpoint-9740           0.322911  0.221483  0.322911   
            monolingual-bloomz-560m   0.288855  0.211677  0.288855   
            pair-bloomz-1b1           0.371155  0.250609  0.371155   
            pair-bloomz-1b1-R-100     0.323170  0.228720  0.323170   
            pair-bloomz-1b1-R-1000    0.277167  0.210304  0.277167   
            pair-bloomz-1b1-R-10000   0.309184  0.229517  0.309184   
            pair-bloomz-560m          0.332636  0.279281  0.332636   
            pair-bloomz-560m-R-100    0.320239  0.233953  0.320239   
            pair-bloomz-560m-R-1000   0.419180  0.326515  0.419180   
            pair-bloomz-560m-R-10000  0.369725  0.282386  0.369725   
            random-bloomz-560m        0.306909  0.253996  0.306909   
            translation-bloomz-560m   0.346337  0.268612  0.346337   

                                      weighted_f1  
prompt_lang model                                  
EN          bilingual-bloomz-560m        0.339234  
            bloomz-1b1                   0.364528  
            bloomz-1b7                   0.380157  
            bloomz-3b                    0.407582  
            bloomz-560m                  0.298149  
            checkpoint-19480             0.228843  
            checkpoint-29220             0.266920  
            checkpoint-38960             0.237882  
            checkpoint-9740              0.282995  
            monolingual-bloomz-560m      0.329202  
            pair-bloomz-1b1              0.304292  
            pair-bloomz-1b1-R-100        0.244145  
            pair-bloomz-1b1-R-1000       0.177899  
            pair-bloomz-1b1-R-10000      0.227768  
            pair-bloomz-560m             0.318037  
            pair-bloomz-560m-R-100       0.237733  
            pair-bloomz-560m-R-1000      0.298426  
            pair-bloomz-560m-R-10000     0.287777  
            random-bloomz-560m           0.322582  
          

### Related Work
#### Instruction-TUning 

#### Bilingual Alignment

#### Continual Learning

### Method
#### Instruct-Augment
- Token-Level language denoising (TLD) -> monolingual sentence completion
- Token-Level language alignment (TLA) -> contextualized sentence completion
- Sentence-level language alignment (SLA) -> machine translation
- In addition, we explore pair language alignment (PLA) method which combination of both contextualized sentence completion and machine translation, and monolingual-bilingual learning (MBL), which is a combination of all three methods.

#### Continual Learning -> Simple Rehearsal
- R-100, R-1000, R-10000

### Experiment
- 2000 parallel data for learning new languages
- We use instruction-tuned model, BLOOMZ, with only 560M parameters due to limited time and resource.
- We cover 6 languages that is not pre-trained on BLOOMZ, i.e., A, B, C, D, E, F.

### Results & Discussion
#### No Continual Learning
- Among the 3 basic methods (TLD, TLA, and SLA), TLA performs the best over all downstream tasks
- In terms of generalization to other languages, <Can-it-helps?>
- We observe forgetting problems on the other pre-learnt languages, i.e. English and Indonesian, we suspect this happens due to catastrophic forgetting

#### With Continual Learning
- 100 and 10,000 fails to improve the performance, but 1,000 improve the performance consistently on almost all tasks
   - Requires balancing between new tasks and old tasks.
- 

### Future Works
- Scale upp to larger size models
- 

# Analyze model per task group

In [7]:
mdf.loc[mdf['task'].str.startswith('xnli_'),:].groupby(['model'])[['acc','macro_f1','micro_f1','weighted_f1']].mean()

,acc,macro_f1,micro_f1,weighted_f1
model,,,,
bilingual-bloomz-560m,0.334527,0.171678,0.334527,0.171678
bloomz-1b1,0.336177,0.178051,0.336177,0.178051
bloomz-1b7,0.354185,0.218703,0.354185,0.218703
bloomz-3b,0.356363,0.221926,0.356363,0.221926
bloomz-560m,0.340306,0.197887,0.340306,0.197887
monolingual-bloomz-560m,0.335338,0.186514,0.335338,0.186514
translation-bloomz-560m,0.334411,0.172372,0.334411,0.172372


In [8]:
mdf.loc[
    (mdf['task'].str.startswith('nusax'))
,:].groupby(['model'])[['acc','macro_f1','micro_f1','weighted_f1']].mean()

,acc,macro_f1,micro_f1,weighted_f1
model,,,,
bilingual-bloomz-560m,0.416007,0.310600,0.416007,0.333499
bloomz-1b1,0.513472,0.379785,0.513472,0.432903
bloomz-1b7,0.502708,0.359108,0.502708,0.405421
bloomz-3b,0.566771,0.421318,0.566771,0.480302
bloomz-560m,0.443611,0.314911,0.443611,0.356240
checkpoint-19480,0.403958,0.253971,0.403958,0.278848
checkpoint-29220,0.395556,0.256838,0.395556,0.289581
checkpoint-38960,0.391979,0.262773,0.391979,0.291583
checkpoint-9740,0.414931,0.282982,0.414931,0.312247


In [9]:
mdf.loc[mdf['task'].str.startswith('nusa_kalimat_emot'),:].groupby(['model'])[['acc','macro_f1','micro_f1','weighted_f1']].mean()

,acc,macro_f1,micro_f1,weighted_f1
model,,,,
bilingual-bloomz-560m,0.195424,0.085927,0.195424,0.083773
bloomz-1b1,0.236643,0.113682,0.236643,0.115983
bloomz-1b7,0.265237,0.144265,0.265237,0.158464
bloomz-3b,0.272423,0.155869,0.272423,0.168327
bloomz-560m,0.239758,0.148607,0.239758,0.155788
checkpoint-19480,0.225729,0.094887,0.225729,0.104563
checkpoint-29220,0.212524,0.098850,0.212524,0.104133
checkpoint-38960,0.208306,0.108513,0.208306,0.111586
checkpoint-9740,0.233941,0.088864,0.233941,0.099745


In [10]:
mdf.loc[mdf['task'].str.startswith('nusa_kalimat_senti'),:].groupby(['model'])[['acc','macro_f1','micro_f1','weighted_f1']].mean()

,acc,macro_f1,micro_f1,weighted_f1
model,,,,
bilingual-bloomz-560m,0.482096,0.418283,0.482096,0.443500
bloomz-1b1,0.574394,0.503596,0.574394,0.544567
bloomz-1b7,0.589540,0.504131,0.589540,0.540276
bloomz-3b,0.646664,0.559592,0.646664,0.626146
bloomz-560m,0.429758,0.420991,0.429758,0.411413
checkpoint-19480,0.344520,0.301603,0.344520,0.239669
checkpoint-29220,0.361694,0.334292,0.361694,0.286725
checkpoint-38960,0.370402,0.350739,0.370402,0.312500
checkpoint-9740,0.397725,0.357907,0.397725,0.323065


In [12]:
mdf.loc[
    (~mdf['task'].str.startswith('xnli')) & (~mdf['task'].str.startswith('nusa')),
:].groupby(['model','prompt_lang'])[['acc','macro_f1','micro_f1','weighted_f1']].mean()

acc  macro_f1  micro_f1  \
model                    prompt_lang                                 
bilingual-bloomz-560m    EN           0.398891  0.278782  0.398891   
                         ID           0.364847  0.264173  0.364847   
bloomz-1b1               EN           0.462512  0.327055  0.462512   
                         ID           0.490545  0.373169  0.490545   
bloomz-1b7               EN           0.455369  0.320702  0.455369   
                         ID           0.510823  0.424169  0.510823   
bloomz-3b                EN           0.464366  0.337186  0.464366   
                         ID           0.518900  0.431164  0.518900   
bloomz-560m              EN           0.455505  0.335674  0.455505   
                         ID           0.452641  0.373856  0.452641   
checkpoint-19480         EN           0.491395  0.368995  0.491395   
                         ID           0.411291  0.318012  0.411291   
checkpoint-29220         EN           0.474319  0.348113  0.474319   
                         ID           0.352612  0.263025  0.352612   
checkpoint-38960         EN           0.437586  0.320335  0.437586   
                         ID           0.370132  0.302237  0.370132   
checkpoint-9740          EN           0.496720  0.374926  0.496720   
                         ID           0.412875  0.312450  0.412875   
monolingual-bloomz-560m  EN           0.390519  0.281032  0.390519   
                         ID           0.333856  0.227488  0.333856   
pair-bloomz-1b1          EN           0.396581  0.239976  0.396581   
                         ID           0.395142  0.263702  0.395142   
pair-bloomz-1b1-R-100    EN           0.335440  0.227114  0.335440   
                         ID           0.366755  0.253831  0.366755   
pair-bloomz-1b1-R-1000   EN           0.367230  0.248485  0.367230   
                         ID           0.366396  0.248493  0.366396   
pair-bloomz-1b1-R-10000  EN           0.432970  0.283261  0.432970   
                         ID           0.419937  0.301875  0.419937   
pair-bloomz-560m         EN           0.401351  0.265536  0.401351   
                         ID           0.375545  0.291487  0.375545   
pair-bloomz-560m-R-100   EN           0.441160  0.321056  0.441160   
                         ID           0.380938  0.306921  0.380938   
pair-bloomz-560m-R-1000  EN           0.433381  0.302152  0.433381   
                         ID           0.491171  0.345382  0.491171   
pair-bloomz-560m-R-10000 EN           0.456995  0.344808  0.456995   
                         ID           0.384086  0.314170  0.384086   
random-bloomz-560m       EN           0.385931  0.274487  0.385931   
                         ID           0.352332  0.268072  0.352332   
translation-bloomz-560m  EN           0.364409  0.229741  0.364409   
                         ID           0.355952  0.282314  0.355952   

                                      weighted_f1  
model                    prompt_lang               
bilingual-bloomz-560m    EN              0.306586  
                         ID              0.273851  
bloomz-1b1               EN              0.366678  
                         ID              0.409193  
bloomz-1b7               EN              0.361813  
                         ID              0.454680  
bloomz-3b                EN              0.376435  
                         ID              0.463574  
bloomz-560m              EN              0.365215  
                         ID              0.402889  
checkpoint-19480         EN              0.411055  
                         ID              0.340416  
checkpoint-29220         EN              0.387698  
                         ID              0.273154  
checkpoint-38960         EN              0.351941  
                         ID              0.317474  
checkpoint-9740          EN              0.418769  
                         ID              0.335288  
monolingual-bloomz-560m  EN              0.307545  
          

### Analyze language per task group

In [14]:
mdf.loc[(mdf['task'].str.startswith('xnli')),:].groupby(
    ['model','prompt_lang','task']
).mean().reset_index().pivot(
    ['model','prompt_lang'],'task', ['acc']
    # ['model','lang'],'task', ['acc', 'macro_f1', 'micro_f1','weighted_f1']
)

acc                                \
task                                  xnli_ar   xnli_bg   xnli_de   xnli_el   
model                   prompt_lang                                           
bilingual-bloomz-560m   EN           0.333267  0.333333  0.333333  0.333267   
bloomz-1b1              EN           0.336194  0.334464  0.334664  0.335729   
bloomz-1b7              EN           0.360546  0.342182  0.353027  0.343713   
bloomz-3b               EN           0.367066  0.343846  0.350499  0.344844   
bloomz-560m             EN           0.337392  0.334265  0.338257  0.336128   
monolingual-bloomz-560m EN           0.335263  0.338989  0.335595  0.336128   
translation-bloomz-560m EN           0.333866  0.333466  0.333599  0.333666   

                                                                             \
task                                  xnli_en   xnli_es   xnli_fr   xnli_hi   
model                   prompt_lang                                           
bilingual-bloomz-560m   EN           0.333333  0.336261  0.333666  0.333200   
bloomz-1b1              EN           0.339920  0.337525  0.337791  0.337591   
bloomz-1b7              EN           0.374983  0.370459  0.368197  0.348237   
bloomz-3b               EN           0.378110  0.378044  0.374917  0.355090   
bloomz-560m             EN           0.347505  0.354890  0.351763  0.337924   
monolingual-bloomz-560m EN           0.333533  0.333533  0.333666  0.336527   
translation-bloomz-560m EN           0.342582  0.333400  0.334597  0.335662   

                                                                             \
task                                  xnli_ru   xnli_sw   xnli_th   xnli_tr   
model                   prompt_lang                                           
bilingual-bloomz-560m   EN           0.333466  0.341118  0.333333  0.333866   
bloomz-1b1              EN           0.334331  0.336061  0.333599  0.333400   
bloomz-1b7              EN           0.348370  0.352229  0.339055  0.339987   
bloomz-3b               EN           0.352761  0.345842  0.339321  0.340785   
bloomz-560m             EN           0.336394  0.335130  0.339787  0.333533   
monolingual-bloomz-560m EN           0.333932  0.336128  0.333333  0.337991   
translation-bloomz-560m EN           0.334597  0.333333  0.333866  0.333333   

                                                                   
task                                  xnli_ur   xnli_vi   xnli_zh  
model                   prompt_lang                                
bilingual-bloomz-560m   EN           0.333400  0.338523  0.334531  
bloomz-1b1              EN           0.335529  0.339920  0.335928  
bloomz-1b7              EN           0.341251  0.369661  0.360878  
bloomz-3b               EN           0.346906  0.364870  0.362542  
bloomz-560m             EN           0.334997  0.345576  0.341051  
monolingual-bloomz-560m EN           0.335529  0.335329  0.334597  
translation-bloomz-560m EN           0.333067  0.333466  0.333666

In [15]:
mdf.loc[(mdf['task'].str.startswith('nusax_senti')),:].groupby(
    ['model','prompt_lang','task']
).mean().reset_index().pivot(
    ['model','prompt_lang'],'task', ['macro_f1']
    # ['model','lang'],'task', ['acc', 'macro_f1', 'micro_f1','weighted_f1']
)

macro_f1                  \
task                                 nusax_senti_ace nusax_senti_ban   
model                    prompt_lang                                   
bilingual-bloomz-560m    EN                 0.315518        0.275554   
                         ID                 0.336973        0.310060   
bloomz-1b1               EN                 0.380609        0.378106   
                         ID                 0.352163        0.353032   
bloomz-1b7               EN                 0.304023        0.305983   
                         ID                 0.371426        0.365934   
bloomz-3b                EN                 0.428186        0.424560   
                         ID                 0.445181        0.420825   
bloomz-560m              EN                 0.274943        0.284812   
                         ID                 0.267869        0.275729   
checkpoint-19480         EN                 0.208513        0.195405   
                         ID                 0.247758        0.254301   
checkpoint-29220         EN                 0.252367        0.244561   
                         ID                 0.208513        0.209177   
checkpoint-38960         EN                 0.252576        0.241480   
                         ID                 0.247945        0.221028   
checkpoint-9740          EN                 0.244527        0.231670   
                         ID                 0.261483        0.255089   
monolingual-bloomz-560m  EN                 0.247412        0.305599   
                         ID                 0.235113        0.244427   
pair-bloomz-1b1          EN                 0.238516        0.232928   
                         ID                 0.251637        0.252923   
pair-bloomz-1b1-R-100    EN                 0.142177        0.134546   
                         ID                 0.212062        0.200822   
pair-bloomz-1b1-R-1000   EN                 0.289515        0.294153   
                         ID                 0.260297        0.270795   
pair-bloomz-1b1-R-10000  EN                 0.206838        0.204904   
                         ID                 0.268500        0.256282   
pair-bloomz-560m         EN                 0.213086        0.247469   
                         ID                 0.282906        0.293864   
pair-bloomz-560m-R-100   EN                 0.236826        0.232660   
                         ID                 0.268158        0.245152   
pair-bloomz-560m-R-1000  EN                 0.303103        0.279637   
                         ID                 0.305135        0.333521   
pair-bloomz-560m-R-10000 EN                 0.247758        0.254927   
                         ID                 0.269159        0.260470   
random-bloomz-560m       EN                 0.293874        0.246474   
                         ID                 0.305860        0.298845   
translation-bloomz-560m  EN                 0.193829        0.194452   
                         ID                 0.249450        0.291400   

                                                                      \
task                                 nusax_senti_bbc nusax_senti_bjn   
model                    prompt_lang                                   
bilingual-bloomz-560m    EN                 0.278880        0.303059   
                         ID                 0.284049        0.262640   
bloomz-1b1               EN                 0.234341        0.392077   
                         ID                 0.286701        0.394590   
bloomz-1b7               EN                 0.190856        0.351441   
                         ID                 0.305545        0.422342   
bloomz-3b                EN                 0.284666        0.485016   
                         ID                 0.340440        0.473319   
bloomz-560m              EN                 0.259356        0.326441   
                         ID                 0.281985        0.279106   
checkpoint-

In [17]:
mdf.loc[(mdf['task'].str.startswith('nusa_kalimat_emot')),:].groupby(
    ['model','prompt_lang','task']
).mean().reset_index().pivot(
    ['model','prompt_lang'],'task', ['macro_f1']
    # ['model','lang'],'task', ['acc', 'macro_f1', 'micro_f1','weighted_f1']
)

macro_f1  \
task                                 nusa_kalimat_emot_abs   
model                    prompt_lang                         
bilingual-bloomz-560m    EN                       0.084318   
                         ID                       0.089178   
bloomz-1b1               EN                       0.069118   
                         ID                       0.182568   
bloomz-1b7               EN                       0.069118   
                         ID                       0.243732   
bloomz-3b                EN                       0.072755   
                         ID                       0.275993   
bloomz-560m              EN                       0.076038   
                         ID                       0.233863   
checkpoint-19480         EN                       0.073987   
                         ID                       0.125228   
checkpoint-29220         EN                       0.069118   
                         ID                       0.132962   
checkpoint-38960         EN                       0.071107   
                         ID                       0.153345   
checkpoint-9740          EN                       0.084424   
                         ID                       0.095725   
monolingual-bloomz-560m  EN                       0.115844   
                         ID                       0.068800   
pair-bloomz-1b1          EN                       0.069118   
                         ID                       0.107578   
pair-bloomz-1b1-R-100    EN                       0.106012   
                         ID                       0.053179   
pair-bloomz-1b1-R-1000   EN                       0.069118   
                         ID                       0.087290   
pair-bloomz-1b1-R-10000  EN                       0.069118   
                         ID                       0.133506   
pair-bloomz-560m         EN                       0.071266   
                         ID                       0.142528   
pair-bloomz-560m-R-100   EN                       0.069160   
                         ID                       0.138488   
pair-bloomz-560m-R-1000  EN                       0.070372   
                         ID                       0.112606   
pair-bloomz-560m-R-10000 EN                       0.117741   
                         ID                       0.164261   
random-bloomz-560m       EN                       0.081481   
                         ID                       0.081693   
translation-bloomz-560m  EN                       0.070351   
                         ID                       0.139520   

                                                            \
task                                 nusa_kalimat_emot_bew   
model                    prompt_lang                         
bilingual-bloomz-560m    EN                       0.073395   
                         ID                       0.097604   
bloomz-1b1               EN                       0.069421   
                         ID                       0.151744   
bloomz-1b7               EN                       0.069421   
                         ID                       0.231712   
bloomz-3b                EN                       0.070554   
                         ID                       0.267299   
bloomz-560m              EN                       0.072969   
                         ID                       0.230298   
checkpoint-19480         EN                       0.073254   
                         ID                       0.121243   
checkpoint-29220         EN                       0.069977   
                         ID                       0.134532   
checkpoint-38960         EN                       0.069460   
                         ID                       0.153945   
checkpoint-9740          EN                       0.081941   
                         ID                       0.100171   
monolingual-bloomz-560m  EN                       0.105538   
                   

In [19]:
mdf.loc[(mdf['task'].str.startswith('nusa_kalimat_senti')),:].groupby(
    ['model','prompt_lang','task']
).mean().reset_index().pivot(
    ['model','prompt_lang'],'task', ['macro_f1']
    # ['model','lang'],'task', ['acc', 'macro_f1', 'micro_f1','weighted_f1']
)

macro_f1  \
task                                 nusa_kalimat_senti_abs   
model                    prompt_lang                          
bilingual-bloomz-560m    EN                        0.522304   
                         ID                        0.352246   
bloomz-1b1               EN                        0.566364   
                         ID                        0.464774   
bloomz-1b7               EN                        0.617435   
                         ID                        0.432954   
bloomz-3b                EN                        0.577350   
                         ID                        0.570595   
bloomz-560m              EN                        0.442492   
                         ID                        0.409556   
checkpoint-19480         EN                        0.386896   
                         ID                        0.237258   
checkpoint-29220         EN                        0.374342   
                         ID                        0.270158   
checkpoint-38960         EN                        0.311914   
                         ID                        0.340936   
checkpoint-9740          EN                        0.374626   
                         ID                        0.279755   
monolingual-bloomz-560m  EN                        0.500275   
                         ID                        0.274517   
pair-bloomz-1b1          EN                        0.444435   
                         ID                        0.405699   
pair-bloomz-1b1-R-100    EN                        0.492581   
                         ID                        0.424465   
pair-bloomz-1b1-R-1000   EN                        0.230483   
                         ID                        0.245109   
pair-bloomz-1b1-R-10000  EN                        0.359428   
                         ID                        0.275520   
pair-bloomz-560m         EN                        0.530972   
                         ID                        0.405509   
pair-bloomz-560m-R-100   EN                        0.338634   
                         ID                        0.317993   
pair-bloomz-560m-R-1000  EN                        0.426575   
                         ID                        0.536796   
pair-bloomz-560m-R-10000 EN                        0.450265   
                         ID                        0.412311   
random-bloomz-560m       EN                        0.533642   
                         ID                        0.373827   
translation-bloomz-560m  EN                        0.265922   
                         ID                        0.398103   

                                                             \
task                                 nusa_kalimat_senti_bew   
model                    prompt_lang                          
bilingual-bloomz-560m    EN                        0.513690   
                         ID                        0.318484   
bloomz-1b1               EN                        0.626431   
                         ID                        0.473891   
bloomz-1b7               EN                        0.629431   
                         ID                        0.440561   
bloomz-3b                EN                        0.610900   
                         ID                        0.581326   
bloomz-560m              EN                        0.471191   
                         ID                        0.417616   
checkpoint-19480         EN                        0.385559   
                         ID                        0.241778   
checkpoint-29220         EN                        0.437647   
                         ID                        0.273575   
checkpoint-38960         EN                        0.368891   
                         ID                        0.352290   
checkpoint-9740          EN                        0.454239   
                         ID                        0.283020   
monolingual-bloomz

### Analyze per task

In [16]:
mdf.groupby(
    ['model','lang','task']
).mean().reset_index().pivot(
    ['model','lang'],'task', ['acc', 'macro_f1', 'micro_f1','weighted_f1']
)

acc                                        \
task                               emot   imdb_jv indolem_sentiment   indonli   
model                    lang                                                   
bilingual-bloomz-560m    EN    0.223485  0.463267          0.643917  0.328381   
                         ID    0.186364  0.500040          0.495219  0.333591   
bloomz-1b1               EN    0.229545  0.402120          0.768546  0.379317   
                         ID    0.299242  0.461000          0.754698  0.345103   
bloomz-1b7               EN    0.231818  0.331960          0.760303  0.409930   
                         ID    0.487879  0.396333          0.636334  0.404528   
bloomz-3b                EN    0.247727  0.317120          0.772173  0.412309   
                         ID    0.411364  0.379587          0.716782  0.464017   
bloomz-560m              EN    0.227273  0.435507          0.702275  0.368705   
                         ID    0.371970  0.474400          0.567755  0.341694   
checkpoint-19480         EN    0.236364       NaN          0.680185  0.354364   
                         ID    0.261364       NaN          0.535443  0.301691   
checkpoint-29220         EN    0.225758       NaN          0.623475  0.354042   
                         ID    0.218939       NaN          0.404880  0.310631   
checkpoint-38960         EN    0.228788       NaN          0.531157  0.357065   
                         ID    0.243182       NaN          0.410155  0.309859   
checkpoint-9740          EN    0.236364       NaN          0.682493  0.352691   
                         ID    0.250758       NaN          0.470491  0.335584   
monolingual-bloomz-560m  EN    0.218939  0.480307          0.614573  0.339572   
                         ID    0.161364  0.497267          0.338279  0.336163   
pair-bloomz-560m         EN    0.227273  0.459160          0.691724  0.338414   
                         ID    0.198485  0.494947          0.509726  0.342401   
pair-bloomz-560m-R-100   EN    0.226515       NaN          0.529838  0.358287   
                         ID    0.238636       NaN          0.458622  0.327159   
pair-bloomz-560m-R-1000  EN    0.228030       NaN          0.581602  0.339893   
                         ID    0.253788       NaN          0.706891  0.338671   
pair-bloomz-560m-R-10000 EN    0.246970       NaN          0.663699  0.346646   
                         ID    0.255303       NaN          0.470162  0.300212   
random-bloomz-560m       EN    0.221970  0.469893          0.601385  0.329153   
                         ID    0.172727  0.498107          0.466535  0.333462   
translation-bloomz-560m  EN    0.226515  0.482280          0.462578  0.339636   
                         ID    0.254545  0.493640          0.489944  0.332755   

                                                                           \
task                          nusa_kalimat_emot_abs nusa_kalimat_emot_bew   
model                    lang                                               
bilingual-bloomz-560m    EN                0.209630              0.206667   
                         ID                0.170370              0.189000   
bloomz-1b1               EN                0.208889              0.210000   
                         ID                0.284444              0.258000   
bloomz-1b7               EN                0.208889              0.210000   
                         ID                0.328889              0.330667   
bloomz-3b                EN                0.211111              0.210667   
                         ID                0.348148              0.347000   
bloomz-560m              EN                0.211852              0.210333   
                         ID                0.280741              0.268667   
checkpoint-19480         EN                0.208148              0.210667   
                         ID                0.242963              0.243333   
checkpoint-29220         EN                0.

In [17]:
mdf.groupby(
    ['model','lang','task']
).mean().reset_index().pivot(
    ['model','lang'],'task', ['acc', 'macro_f1', 'micro_f1','weighted_f1']
).to_csv('result_zero_shot.csv', index=True)

In [21]:
dfs = []
for path in glob.glob('../metrics/*.csv'):
    lang, model = path[:-4].split('_')[2:4]
    df = pd.read_csv(path)
    df['lang'] = lang[:2]
    df['model'] = model
    dfs.append(df)
pd.concat(dfs).groupby(['model', 'lang']).mean()

accuracy  f1_score
model                   lang                    
bilingual-bloomz-560m   EN    0.410617  0.294240
                        ID    0.326184  0.250878
bloomz-1b1              EN    0.412547  0.292133
                        ID    0.455926  0.337494
bloomz-1b7              EN    0.442906  0.299240
                        ID    0.440505  0.354371
bloomz-3b               EN    0.453281  0.317125
                        ID    0.503425  0.408722
bloomz-560m             EN    0.376052  0.264769
                        ID    0.381165  0.318804
checkpoint-19480        EN    0.357405  0.241843
                        ID    0.322891  0.220424
checkpoint-29220        EN    0.369752  0.266411
                        ID    0.299560  0.210899
checkpoint-38960        EN    0.351368  0.246737
                        ID    0.316262  0.250641
checkpoint-9740         EN    0.391128  0.278679
                        ID    0.332381  0.231059
monolingual-bloomz-560m EN    0.395751  0.286098
                        ID    0.295605  0.214048
pair-bloomz-1b1         EN    0.352720  0.228751
                        ID    0.330202  0.235365
pair-bloomz-560m        EN    0.388049  0.268916
                        ID    0.365575  0.284204
random-bloomz-560m      EN    0.390464  0.288644
                        ID    0.313723  0.256107
translation-bloomz-560m EN    0.319093  0.196509
                        ID    0.347779  0.270667

# 32-bit

In [19]:
metrics = {
    'task': [], 'lang': [], 'prompt': [], 'model': [], 'acc': [],
    'macro_f1': [], 'macro_prec': [], 'macro_rec': [],
    'micro_f1': [], 'micro_prec': [], 'micro_rec': []
}
for path in glob.glob('outputs_32bit/*.csv'):
    if '_nusantara_text_' in path:
        task, meta = path.split('/')[-1].split('_nusantara_text_')
    else:
        task, meta = path.split('/')[-1].split('_nusantara_pairs_')
    prompt, model = meta[:-4].split('_')
    
    df = pd.read_csv(path)
    cls_report = classification_report(df['Gold'], df['Pred'], output_dict=True)
    micro_f1, micro_prec, micro_rec, _ = precision_recall_fscore_support(df['Gold'], df['Pred'], average='micro')

    metrics['task'].append(task)
    metrics['lang'].append(prompt[:2])
    metrics['prompt'].append(prompt)
    metrics['model'].append(model)
    metrics['acc'].append(cls_report['accuracy'])
    metrics['macro_f1'].append(cls_report['macro avg']['f1-score'])
    metrics['macro_prec'].append(cls_report['macro avg']['precision'])
    metrics['macro_rec'].append(cls_report['macro avg']['recall'])
    metrics['micro_f1'].append(micro_f1)
    metrics['micro_prec'].append(micro_prec)
    metrics['micro_rec'].append(micro_rec)

/home/samuel/anaconda2/envs/env_nusa_exp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samuel/anaconda2/envs/env_nusa_exp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samuel/anaconda2/envs/env_nusa_exp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [20]:
pd.DataFrame(metrics).groupby(['model','lang']).mean()

acc  macro_f1  macro_prec  macro_rec  micro_f1  \
model       lang                                                        
bloomz-1b1  EN    0.522649  0.379011    0.404832   0.457861  0.522649   
            ID    0.528781  0.392760    0.464153   0.466697  0.528781   
bloomz-1b7  EN    0.514377  0.368960    0.419022   0.451026  0.514377   
            ID    0.526789  0.411978    0.508149   0.480274  0.526789   
bloomz-560m EN    0.475819  0.332466    0.379742   0.431091  0.475819   
            ID    0.443664  0.297815    0.377854   0.402763  0.443664   
mt0-base    EN    0.380048  0.226484    0.218003   0.358409  0.380048   
            ID    0.373831  0.231813    0.253617   0.353473  0.373831   
mt0-large   EN    0.426327  0.293602    0.403439   0.397829  0.426327   
            ID    0.379830  0.233884    0.230554   0.359814  0.379830   

                  micro_prec  micro_rec  
model       lang                         
bloomz-1b1  EN      0.522649   0.522649  
            ID      0.528781   0.528781  
bloomz-1b7  EN      0.514377   0.514377  
            ID      0.526789   0.526789  
bloomz-560m EN      0.475819   0.475819  
            ID      0.443664   0.443664  
mt0-base    EN      0.380048   0.380048  
            ID      0.373831   0.373831  
mt0-large   EN      0.426327   0.426327  
            ID      0.379830   0.379830

In [5]:
def get_logprobs(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to('cuda')
    input_ids, output_ids = inputs["input_ids"], inputs["input_ids"][:, 1:]
    
    outputs = model(**inputs, labels=input_ids)
    logits = outputs.logits
    
    logprobs = torch.gather(F.log_softmax(logits, dim=2), 2, output_ids.unsqueeze(2))
    
    return logprobs.mean()

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained('bigscience/mt0-base')
tokenizer = AutoTokenizer.from_pretrained('bigscience/mt0-base')

In [ ]:
inputs = tokenizer('The emotion of "I am really glad that you are here. Thank you!" is', return_tensors='pt')
input_ids, output_ids = inputs["input_ids"], inputs["input_ids"][:, 1:]
outputs = model(**inputs, labels=input_ids)
x1 = F.log_softmax(outputs.logits, dim=2)
y1 = torch.gather(F.log_softmax(outputs.logits, dim=2), 2, output_ids.unsqueeze(2)).mean()

In [ ]:
model = AutoModelForCausalLM.from_pretrained('bigscience/bloomz-560m')
tokenizer = AutoTokenizer.from_pretrained('bigscience/mt0-base')

In [43]:
pd.concat(dfs).groupby(
    ['model','lang','index']
).mean().reset_index().pivot(
    ['model','lang'], 'index', ['accuracy', 'f1_score']
).to_csv('result_zero_shot.csv', index=True)

# Final Project

In [7]:
fp_df = mdf.loc[
    (~mdf['task'].str.startswith('xnli')) & ~mdf['task'].isin(['su_emot', 'indonli', 'indolem_sentiment', 'smsa', 'emot', 'imdb_jv'])
]

In [12]:
fp_df.sort_values(['task', 'model']).head(30)

,task,task_lang,prompt_lang,prompt,model,acc,macro_f1,macro_prec,macro_rec,micro_f1,micro_prec,micro_rec,weighted_f1,weighted_prec,weighted_rec
526,nusa_kalimat_emot_abs,abs,ID,ID3,bilingual-bloomz-560m,0.157778,0.083115,0.079770,0.205731,0.157778,0.157778,0.157778,0.076699,0.084122,0.157778
671,nusa_kalimat_emot_abs,abs,EN,EN,bilingual-bloomz-560m,0.208889,0.103098,0.152474,0.208018,0.208889,0.208889,0.208889,0.097275,0.158016,0.208889
1193,nusa_kalimat_emot_abs,abs,ID,ID,bilingual-bloomz-560m,0.146667,0.062306,0.100973,0.203905,0.146667,0.146667,0.146667,0.050782,0.111085,0.146667
1356,nusa_kalimat_emot_abs,abs,EN,EN2,bilingual-bloomz-560m,0.211111,0.080739,0.108559,0.204222,0.211111,0.211111,0.211111,0.080334,0.090420,0.211111
2107,nusa_kalimat_emot_abs,abs,EN,EN3,bilingual-bloomz-560m,0.208889,0.069118,0.041778,0.200000,0.208889,0.208889,0.208889,0.072190,0.043635,0.208889
2260,nusa_kalimat_emot_abs,abs,ID,ID2,bilingual-bloomz-560m,0.206667,0.122115,0.095712,0.236241,0.206667,0.206667,0.206667,0.123576,0.102352,0.206667
0,nusa_kalimat_emot_abs,abs,EN,EN2,bloomz-1b1,0.208889,0.069118,0.041778,0.200000,0.208889,0.208889,0.208889,0.072190,0.043635,0.208889
1611,nusa_kalimat_emot_abs,abs,ID,ID2,bloomz-1b1,0.233333,0.148112,0.113794,0.250385,0.233333,0.233333,0.233333,0.130836,0.098166,0.233333
1755,nusa_kalimat_emot_abs,abs,ID,ID3,bloomz-1b1,0.320000,0.218037,0.413753,0.290312,0.320000,0.320000,0.320000,0.222516,0.454712,0.320000
2034,nusa_kalimat_emot_abs,abs,EN,EN,bloomz-1b1,0.208889,0.069118,0.041778,0.200000,0.208889,0.208889,0.208889,0.072190,0.043635,0.208889


In [13]:
fp_df.groupby(['task', 'model']).size()

task                   model                   
nusa_kalimat_emot_abs  bilingual-bloomz-560m       6
                       bloomz-1b1                  6
                       bloomz-1b7                  6
                       bloomz-3b                   6
                       bloomz-560m                 6
                                                  ..
nusax_senti_sun        pair-bloomz-560m-R-100      6
                       pair-bloomz-560m-R-1000     6
                       pair-bloomz-560m-R-10000    6
                       random-bloomz-560m          6
                       translation-bloomz-560m     6
Length: 544, dtype: int64

### Per Task Result

### Catastrophic Forgetting

In [45]:
cf_models = ['pair-bloomz-560m','checkpoint-9740','checkpoint-19480', 'checkpoint-29220', 'checkpoint-38960']

In [46]:
cf_df = fp_df.loc[fp_df['model'].isin(cf_models)]

In [47]:
cf_df

,task,lang,prompt,model,acc,macro_f1,macro_prec,macro_rec,micro_f1,micro_prec,micro_rec,weighted_f1,weighted_prec,weighted_rec
1,nusax_senti_ace,ID,ID3,checkpoint-29220,0.337500,0.225947,0.221954,0.297321,0.337500,0.337500,0.337500,0.256942,0.252985,0.337500
5,nusa_kalimat_emot_mui,ID,ID3,checkpoint-19480,0.177778,0.102929,0.077812,0.200319,0.177778,0.177778,0.177778,0.102981,0.081851,0.177778
11,nusa_kalimat_senti_jav,ID,ID2,checkpoint-9740,0.342500,0.318982,0.509669,0.504498,0.342500,0.342500,0.342500,0.266883,0.597522,0.342500
19,nusa_kalimat_emot_mad,ID,ID,checkpoint-38960,0.240000,0.218514,0.242729,0.250257,0.240000,0.240000,0.240000,0.217395,0.250623,0.240000
21,nusa_kalimat_emot_btk,EN,EN2,checkpoint-19480,0.209000,0.069148,0.041842,0.199048,0.209000,0.209000,0.209000,0.072605,0.043934,0.209000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,nusa_kalimat_emot_bew,EN,EN2,checkpoint-29220,0.210000,0.069421,0.042000,0.200000,0.210000,0.210000,0.210000,0.072893,0.044100,0.210000
4049,nusa_kalimat_senti_btk,EN,EN,checkpoint-29220,0.419167,0.418872,0.516633,0.515850,0.419167,0.419167,0.419167,0.413490,0.604494,0.419167
4057,nusa_kalimat_senti_bhp,EN,EN,checkpoint-9740,0.534000,0.483029,0.492370,0.490873,0.534000,0.534000,0.534000,0.554453,0.589801,0.534000
4059,nusa_kalimat_senti_rej,EN,EN2,checkpoint-38960,0.446000,0.445501,0.510793,0.511339,0.446000,0.446000,0.446000,0.451955,0.587439,0.446000
